In [11]:
# https://velog.io/@jihyunko/python-Selenium%EA%B3%BC-Beautifulsoup%EB%A5%BC-%ED%99%9C%EC%9A%A9%ED%95%9C-%EC%9C%A0%ED%8A%9C%EB%B8%8C-%EB%8C%93%EA%B8%80-%ED%81%AC%EB%A1%A4%EB%A7%81
from selenium import webdriver
import time
from openpyxl import Workbook
import pandas as pd
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

import warnings 
warnings.filterwarnings('ignore')

In [25]:
from selenium.webdriver.common.by import By

In [48]:
from selenium.webdriver.support.ui import WebDriverWait

In [50]:
from selenium.webdriver.support import expected_conditions as EC

In [54]:
wb = Workbook(write_only=True)
ws = wb.create_sheet()

driver = webdriver.Chrome()
driver.get("https://www.youtube.com/watch?v=xShT8ZheaCA")
driver.implicitly_wait(3)

time.sleep(1.5)

driver.execute_script("window.scrollTo(0, 800)")
time.sleep(3)

In [55]:
last_height = driver.execute_script("return document.documentElement.scrollHeight")

while True:
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(1.5)

    new_height = driver.execute_script("return document.documentElement.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

time.sleep(1.5)

In [56]:
try:
    driver.find_element(By.XPATH, "//*[@id='dismiss-button']/yt-button-shape/button/yt-touch-feedback-shape/div/div[2]").click()
except:
    pass

In [47]:
buttons = driver.find_elements(By.CLASS_NAME, "yt-spec-touch-feedback-shape__fill")

time.sleep(1.5)

for button in buttons:
    button.send_keys(Keys.ENTER)
    time.sleep(1.5)
    button.click()

ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=128.0.6613.138)
Stacktrace:
	GetHandleVerifier [0x00007FF6D7069412+29090]
	(No symbol) [0x00007FF6D6FDE239]
	(No symbol) [0x00007FF6D6E9B009]
	(No symbol) [0x00007FF6D6EE8843]
	(No symbol) [0x00007FF6D6EE6A4B]
	(No symbol) [0x00007FF6D6F166EA]
	(No symbol) [0x00007FF6D6EE26C6]
	(No symbol) [0x00007FF6D6F16900]
	(No symbol) [0x00007FF6D6F365A2]
	(No symbol) [0x00007FF6D6F16493]
	(No symbol) [0x00007FF6D6EE09D1]
	(No symbol) [0x00007FF6D6EE1B31]
	GetHandleVerifier [0x00007FF6D738871D+3302573]
	GetHandleVerifier [0x00007FF6D73D4243+3612627]
	GetHandleVerifier [0x00007FF6D73CA417+3572135]
	GetHandleVerifier [0x00007FF6D7125EB6+801862]
	(No symbol) [0x00007FF6D6FE945F]
	(No symbol) [0x00007FF6D6FE4FB4]
	(No symbol) [0x00007FF6D6FE5140]
	(No symbol) [0x00007FF6D6FD461F]
	BaseThreadInitThunk [0x00007FFB0320257D+29]
	RtlUserThreadStart [0x00007FFB04E8AF28+40]


In [51]:
# 대댓글 모두 열기
try:
    buttons = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CLASS_NAME, "yt-spec-touch-feedback-shape__fill"))
    )

    for button in buttons:
        try:
            driver.execute_script("arguments[0].scrollIntoView();", button)  # 버튼이 화면에 보이도록 스크롤
            time.sleep(1)  # 페이지 로딩 대기
            clickable_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(button))
            clickable_button.click()
            time.sleep(1.5)  # 대댓글이 열리는 시간 대기
        except Exception as e:
            print(f"Error clicking button: {e}")
except Exception as e:
    print(f"Error finding buttons: {e}")

time.sleep(1.5)

Error clicking button: Message: 

Error clicking button: Message: 

Error clicking button: Message: 

Error clicking button: Message: 

Error clicking button: Message: 

Error clicking button: Message: 

Error clicking button: Message: 

Error clicking button: Message: 

Error clicking button: Message: 

Error clicking button: Message: 



KeyboardInterrupt: 

In [45]:
html_source = driver.page_source
soup = BeautifulSoup(html_source, 'html.parser')

id_list = soup.select("ytd-comment-thread-renderer #author-text")
comment_list = soup.select("ytd-comment-thread-renderer #content-text")

id_final = []
comment_final = []

for i in range(len(comment_list)):
    temp_id = id_list[i].text
    temp_id = temp_id.replace('\n', '')
    temp_id = temp_id.replace('\t', '')
    temp_id = temp_id.replace('    ', '')
    id_final.append(temp_id) # 댓글 작성자

    temp_comment = comment_list[i].text
    temp_comment = temp_comment.replace('\n', '')
    temp_comment = temp_comment.replace('\t', '')
    temp_comment = temp_comment.replace('    ', '')
    comment_final.append(temp_comment) # 댓글 내용

In [46]:
pd_data = {"아이디" : id_final , "댓글 내용" : comment_final}
youtube_pd = pd.DataFrame(pd_data)

youtube_pd.to_excel('result.xlsx')

In [58]:
# WebDriver 종료
driver.quit()